# Pandas Group By: split, apply & combine

Un 'group by' realiza el siguiente proceso
<ol> 1. Split: dividr los datos en grupos basados en algún criterio </ol>
<ol> 2. Apply: aplicar una función a cada grupo de manera independiente </ol>
<ol> 3. Combine: juntar los resultados en una estructura de datos </ol>

(1) La parte de dividir en grupos es muy directa.

 (2) En el paso de apply podríamos querer hacer alguna de las siguientes operaciones:
<ol> 2.a Agregar: Calcular una estadística de cada grupo (Ej. Suma, promedio) </ol>
<ol> 2.b Transformar: Hacer algunas operaciones específicas a cada grupo y obtener una tabla indexada (Ej. Rellenas NAs para cada grupo según su valor)</ol>
<ol> 2.c Filtrar: Discartar algunos grupos de acuerdo a cómo evalue una operación por cada grupo (ej. descartar datos de grupos que sólo contienen pocos miembros) </ol>

(3) Combinar se refiere al paso interno que hace pandas para regresarnos una sola estructura de datos una vez filtrada

## Dividir un objeto en grupos

In [3]:
import pandas as pd
import numpy as np

In [7]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                           'foo', 'bar', 'foo', 'foo'],
                    'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                    'C' : np.random.randn(8),
                    'D' : np.random.randn(8)})

In [8]:
df

,A,B,C,D
0,foo,one,-0.762482,-0.076724
1,bar,one,0.635572,-0.348381
2,foo,two,-0.230718,0.039220
3,bar,three,-0.460273,-1.212230
4,foo,two,-0.797393,-0.512202
5,bar,two,-0.550724,0.580034
6,foo,one,-1.620649,-0.877534
7,foo,three,-1.282171,-1.233315


En este dataset podríamos agrupar por las columnas A o B o ambas

In [10]:
df_agrupado = df.groupby('A')
df_agrupado

In [12]:
df_agrupadoAB = df.groupby(['A','B'])
df_agrupadoAB

In [33]:
df_agrupadoAB.first()

C         D
A   B                        
bar one    0.635572 -0.348381
    three -0.460273 -1.212230
    two   -0.550724  0.580034
foo one   -0.762482 -0.076724
    three -1.282171 -1.233315
    two   -0.230718  0.039220

esto nos divide el DataFrame basado en las filas. Si quieremos también podemos dividir por columnas utilizando la opción axis=1

In [14]:
def obtener_tipo_de_letra(letra):
    if letra.lower() in 'aeiou':
        return 'vocal'
    else:
        return 'consonant'

In [15]:
df_agrupado_letra = df.groupby(obtener_tipo_de_letra,axis=1) 

In [34]:
df_agrupado_letra.first()

,consonant,vocal
0,one,foo
1,one,bar
2,two,foo
3,three,bar
4,two,foo
5,two,bar
6,one,foo
7,three,foo


Si tenemos dos grupos que comparten un índice, estos se pueden considerar un grupo y agregarse

In [26]:
mi_lista_index = [1,2,3,1,2,3]

In [27]:
s = pd.Series([1,2,3,10,20,30],mi_lista_index)
s

1     1
2     2
3     3
1    10
2    20
3    30
dtype: int64

In [28]:
grouped = s.groupby(level=0)
grouped

In [29]:
grouped.first()

1    1
2    2
3    3
dtype: int64

In [30]:
grouped.last()

1    10
2    20
3    30
dtype: int64

In [31]:
grouped.sum()

1    11
2    22
3    33
dtype: int64

## Ordenar

In [36]:
df2 = pd.DataFrame({'X' : ['B', 'B', 'A', 'A'], 'Y' : [1, 2, 3, 4]})
df2

,X,Y
0,B,1
1,B,2
2,A,3
3,A,4


In [38]:
df2.groupby('X').sum()

,Y
X,
A,7
B,3


In [39]:
df2.groupby('X',sort=False).sum()

,Y
X,
B,3
A,7


Aquí vimos que 'groupby' por default preserva el orden de las observaciones dentro de cada grupo. Asimismo, podemos pedirle a groupby que nos muestre los grupos de observaciones por seperado

In [41]:
df3 = pd.DataFrame({'X' : ['A', 'B', 'A', 'B'], 'Y' : [1, 4, 3, 2]})
df3

,X,Y
0,A,1
1,B,4
2,A,3
3,B,2


In [43]:
df3.groupby('X').get_group('A')

,X,Y
0,A,1
2,A,3


In [44]:
df3.groupby('X').get_group('B')

,X,Y
1,B,4
3,B,2


## Atributos de un groupby

El atributo 'groups' es un diccinario cuyas llaves son los grupos únicos y que corresponden a los valores en las etiquetas de cada grupo

In [45]:
df.groupby('A').groups

{'bar': Int64Index([1, 3, 5], dtype='int64'),
 'foo': Int64Index([0, 2, 4, 6, 7], dtype='int64')}

In [46]:
df.groupby(obtener_tipo_de_letra,axis=1).groups

{'consonant': Index(['B', 'C', 'D'], dtype='object'),
 'vocal': Index(['A'], dtype='object')}

Podemos ver que la función len() nos da el tamaño de estos diccionarios

In [47]:
df_agrupadoAB.groups

{('bar', 'one'): Int64Index([1], dtype='int64'),
 ('bar', 'three'): Int64Index([3], dtype='int64'),
 ('bar', 'two'): Int64Index([5], dtype='int64'),
 ('foo', 'one'): Int64Index([0, 6], dtype='int64'),
 ('foo', 'three'): Int64Index([7], dtype='int64'),
 ('foo', 'two'): Int64Index([2, 4], dtype='int64')}

In [48]:
len(df_agrupadoAB)

6

### Groupby con múltiples índices

In [52]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
         ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
arrays

[['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
 ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

In [53]:
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
index

MultiIndex(levels=[['bar', 'baz', 'foo', 'qux'], ['one', 'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['first', 'second'])

In [54]:
s = pd.Series(np.random.randn(8), index=index)
s

first  second
bar    one      -1.334908
       two       1.561479
baz    one       0.220704
       two      -1.663708
foo    one      -0.145461
       two      -0.105580
qux    one      -0.436897
       two       0.938429
dtype: float64

Entonces podríamos agrupar por uno de los niveles (levels) en la serie

In [56]:
agrupado_s = s.groupby(level=0)
agrupado_s.sum()

first
bar    0.226571
baz   -1.443004
foo   -0.251041
qux    0.501532
dtype: float64

Si el multi-index tiene nombres específicados estos se pueden pasar en vez del número de nivel. Ejemplo:

In [57]:
s.groupby(level='second').sum()

second
one   -1.696561
two    0.730619
dtype: float64

También podemos agregar con múltiples niveles

In [59]:
s

first  second
bar    one      -1.334908
       two       1.561479
baz    one       0.220704
       two      -1.663708
foo    one      -0.145461
       two      -0.105580
qux    one      -0.436897
       two       0.938429
dtype: float64

In [61]:
s.groupby(level=['first','second']).sum()

first  second
bar    one      -1.334908
       two       1.561479
baz    one       0.220704
       two      -1.663708
foo    one      -0.145461
       two      -0.105580
qux    one      -0.436897
       two       0.938429
dtype: float64

### Groupby con múltiples índices: DataFrames

Un DataFrame se puede agrupar como una combinación de las columnas e índices de nivel (levels) especificando los nombres de las columnas como strings y los índices de nivel (levels) como objetos del tipo pd.Grouper

In [21]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]

In [22]:
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
index

MultiIndex(levels=[['bar', 'baz', 'foo', 'qux'], ['one', 'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['first', 'second'])

In [23]:
df = pd.DataFrame({'A': [1, 1, 1, 1, 2, 2, 3, 3],
                    'B': np.arange(8)},
                    index=index)
df

A  B
first second      
bar   one     1  0
      two     1  1
baz   one     1  2
      two     1  3
foo   one     2  4
      two     2  5
qux   one     3  6
      two     3  7

Podemos entonces agrupar el DataFrame por el segundo nivel de índice (level=1) y la columna 'A'

In [12]:
df.groupby([pd.Grouper(level=1), 'A']).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

Alternativamente, podemos establecer los levels por nombre

In [13]:
df.groupby([pd.Grouper(level='second'), 'A']).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

### Iterando Grupos

Regresemos al DataFrame que definimos hace un rato

In [33]:
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                           'foo', 'bar', 'foo', 'foo'],
                    'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                    'C' : np.random.randn(8),
                    'D' : np.random.randn(8)})
df

,A,B,C,D
0,foo,one,-0.339725,0.794186
1,bar,one,-0.208612,0.854498
2,foo,two,0.453653,0.572546
3,bar,three,0.255623,-0.775145
4,foo,two,-0.163829,-0.010360
5,bar,two,-0.198359,0.438740
6,foo,one,-1.678685,-0.277674
7,foo,three,-0.194547,1.469778


In [34]:
grouped = df.groupby('A')
grouped

Podemos ver el índice y grupos

In [35]:
for name, group in grouped:
        print(name)
        print(group)

bar
     A      B         C         D
1  bar    one -0.208612  0.854498
3  bar  three  0.255623 -0.775145
5  bar    two -0.198359  0.438740
foo
     A      B         C         D
0  foo    one -0.339725  0.794186
2  foo    two  0.453653  0.572546
4  foo    two -0.163829 -0.010360
6  foo    one -1.678685 -0.277674
7  foo  three -0.194547  1.469778


mientras que aquí el índice es una tupla y vemos de nuevo los grupos de acuerdo a esta nueva clasificación

In [36]:
for name, group in df.groupby(['A', 'B']):
        print(name)
        print(group)

('bar', 'one')
     A    B         C         D
1  bar  one -0.208612  0.854498
('bar', 'three')
     A      B         C         D
3  bar  three  0.255623 -0.775145
('bar', 'two')
     A    B         C        D
5  bar  two -0.198359  0.43874
('foo', 'one')
     A    B         C         D
0  foo  one -0.339725  0.794186
6  foo  one -1.678685 -0.277674
('foo', 'three')
     A      B         C         D
7  foo  three -0.194547  1.469778
('foo', 'two')
     A    B         C         D
2  foo  two  0.453653  0.572546
4  foo  two -0.163829 -0.010360


### Seleccionando desde un grupo

Un grupo se puede seleccionar de acuerdo a su nombre como si fuera una llave 

In [39]:
grouped.get_group('bar')

,A,B,C,D
1,bar,one,-0.208612,0.854498
3,bar,three,0.255623,-0.775145
5,bar,two,-0.198359,0.438740


Asimismo, se puede seleccionar por múltiples columnas de la siguiente forma

In [41]:
df.groupby(['A', 'B']).get_group(('bar', 'one'))

,A,B,C,D
1,bar,one,-0.208612,0.854498


### Aggregation

Podemos agregar una columna

In [42]:
agrupado = df.groupby('A')

In [43]:
agrupado.aggregate(np.sum)

,C,D
A,,
bar,-0.151348,0.518093
foo,-1.923132,2.548476


o varias columnas a la vez

In [46]:
agrupadoAB = df.groupby(['A', 'B'])

In [47]:
agrupadoAB.aggregate(np.sum)

C         D
A   B                        
bar one   -0.208612  0.854498
    three  0.255623 -0.775145
    two   -0.198359  0.438740
foo one   -2.018409  0.516512
    three -0.194547  1.469778
    two    0.289824  0.562187

Por default estos DF agrupados regresan un DF con multi-index. Este comportamiento se puede modificar utilizando el argumento as_index:

In [48]:
grouped = df.groupby(['A', 'B'], as_index=False)

In [49]:
grouped.aggregate(np.sum)

,A,B,C,D
0,bar,one,-0.208612,0.854498
1,bar,three,0.255623,-0.775145
2,bar,two,-0.198359,0.438740
3,foo,one,-2.018409,0.516512
4,foo,three,-0.194547,1.469778
5,foo,two,0.289824,0.562187


In [50]:
df.groupby('A', as_index=False).sum()

,A,C,D
0,bar,-0.151348,0.518093
1,foo,-1.923132,2.548476


Otra función común es contar el tamaño de cada grupo lo cual se logra con size()

In [52]:
grouped.size()

A    B    
bar  one      1
     three    1
     two      1
foo  one      2
     three    1
     two      2
dtype: int64

también podemos obtener un resumen de varias medidas estadísticas con describe()

In [56]:
grouped.describe().transpose()

0                                                                     1  \
  count      mean std       min       25%       50%       75%       max count   
C   1.0 -0.208612 NaN -0.208612 -0.208612 -0.208612 -0.208612 -0.208612   1.0   
D   1.0  0.854498 NaN  0.854498  0.854498  0.854498  0.854498  0.854498   1.0   

               ...            4               5                                \
       mean    ...          75%       max count      mean       std       min   
C  0.255623    ...    -0.194547 -0.194547   2.0  0.144912  0.436626 -0.163829   
D -0.775145    ...     1.469778  1.469778   2.0  0.281093  0.412177 -0.010360   

                                           
        25%       50%       75%       max  
C -0.009459  0.144912  0.299282  0.453653  
D  0.135367  0.281093  0.426820  0.572546  

[2 rows x 48 columns]

Aplicar varias funciones al mismo tiempo

### Aplicar varias funciones al mismo tiempo

In [57]:
grouped = df.groupby('A')

In [58]:
grouped['C'].agg([np.sum, np.mean, np.std])

,sum,mean,std
A,,,
bar,-0.151348,-0.050449,0.265116
foo,-1.923132,-0.384626,0.784888


In [59]:
grouped.agg([np.sum, np.mean, np.std])

C                             D                    
          sum      mean       std       sum      mean       std
A                                                              
bar -0.151348 -0.050449  0.265116  0.518093  0.172698  0.846769
foo -1.923132 -0.384626  0.784888  2.548476  0.509695  0.688667

Los nombres de las columnas los toma según la función que se aplica, pero sin problema los podemos modificar con .rename

In [60]:
(grouped['C'].agg([np.sum, np.mean, np.std])
              .rename(columns={'sum': 'suma',
                               'mean': 'media',
                               'std': 'desv estándar'})
)

,suma,media,desv estándar
A,,,
bar,-0.151348,-0.050449,0.265116
foo,-1.923132,-0.384626,0.784888


In [62]:
(grouped.agg([np.sum, np.mean, np.std])
         .rename(columns={'sum': 'suma',
                         'mean': 'media',
                         'std': 'desv estándar'})
)

C                                 D                        
         suma     media desv estándar      suma     media desv estándar
A                                                                      
bar -0.151348 -0.050449      0.265116  0.518093  0.172698      0.846769
foo -1.923132 -0.384626      0.784888  2.548476  0.509695      0.688667